# Simple Layer and CI Operations

In [4]:
import os
import omnikeeper_client as okc

create a omnikeeper client


In [5]:
okapiclient = okc.OkApiClient(
    backend_url=os.getenv('OMNIKEEPER_URL'),
    client_id=os.getenv('OMNIKEEPER_AUTH_CLIENTID'),
    username=os.getenv('OMNIKEEPER_AUTH_USERNAME'),
    password=os.getenv('OMNIKEEPER_AUTH_PASSWORD'),
)

now create a layer for holding our attributes:
- create/name it
- set description and color it
- clear all data from layer (if it already exists)

those steps are idempotent since we use upsert and create_layer does not throw an error if it already existed


In [6]:
target_layer = "testlayer_ng"

# create layer, if it does not exist and set layer data
result = okc.create_layer(okapiclient, target_layer)
print(f"create layer ok: {result}")

# set description and color of layer
result = okc.update_layerdata(okapiclient, target_layer, "description done with ng functions", okc.hexString2RGBColor("#353973"))
print(f"update layer ok: {result}")

# # empty layer, if anything is inside
result = okc.truncate_layer(okapiclient, target_layer)
print(f"truncate layer ok: {result}")


create layer ok: True
update layer ok: True
truncate layer ok: True


create a named CI and store attributes to it

In [7]:
# create a single CI with a name attribute
ciid1 = okc.create_ci(okapiclient, "test-ci11", target_layer)
print (f"CI id={ciid1} created")

CI id=1ce49584-d0ca-4f93-a2e6-d931d151aa58 created


In [21]:
# insert/update some attributes
python_native_attributes = {
    "author": "Markus Hof",
}

okc.mutate_ci(okapiclient, target_layer, ciid1, python_native_attributes)

{'mutateCIs': {'affectedCIs': [{'id': '1ce49584-d0ca-4f93-a2e6-d931d151aa58'}]}}

In [22]:
# TODO mcsuk this does not work!
# insert/update some attributes
python_native_attributes = {
    "author": "Maximilian Csuk",
    "is_this_good": True,
}

okc.mutate_ci(okapiclient, target_layer, ciid1, python_native_attributes)


{'message': "Variable '$insertAttributes[1].value.values[0]' is invalid. Unable to convert 'True' to 'String'", 'locations': [{'line': 1, 'column': 57}], 'exception': "System.InvalidOperationException: Unable to convert 'True' to the scalar type 'String'\n   at GraphQL.Types.ScalarGraphType.ThrowValueConversionError(Object value) in /_/src/GraphQL/Types/Scalars/ScalarGraphType.cs:line 235\n   at GraphQL.Types.StringGraphType.ParseValue(Object value) in /_/src/GraphQL/Types/Scalars/StringGraphType.cs:line 23\n   at GraphQL.Validation.ValidationContext.<GetVariableValueAsync>g__ParseValueScalar|57_1(ScalarGraphType scalarGraphType, GraphQLVariableDefinition variableDef, VariableName variableName, Object value) in /_/src/GraphQL/Validation/ValidationContext.cs:line 266"}


False

This CI should appear now in technical interface, mutate it there and check your modified attributes using this code:

In [23]:
okc.get_attributes_of_cis(okapiclient, [target_layer], [ciid1])



{'1ce49584-d0ca-4f93-a2e6-d931d151aa58': {'__name': 'test-ci11',
  'author': 'Markus Hof'}}